In [1]:
import drawSvg as draw
from enum import Enum
import math
strip =[]
stripwidth = 10
striplength=0
def render (strip, striplength, stripwidth):
    d = draw.Drawing(1000,stripwidth)
    for element in strip:
        if type(element) is Plain:
            striplength += element.length
    d.append(draw.Rectangle(0, 0, striplength, stripwidth,stroke='red', fill='none'))        
    d.saveSvg('test.svg')
    d.rasterize()
    return d

In [2]:
class Plain:
    def __init__(self, length):
        self.length = length
def addPlain(strip,len):
    strip.append(Plain(len))
    return strip

strip = addPlain(strip,100)
render (strip, striplength, stripwidth)

In [3]:
class Fold:
    def __init__(self, mv, angle):
        self.mv = mv
        self.angle = angle
    def getLength(self,stripwidth):
        return stripwidth*math.tan(math.radians(90-self.angle))
class MV(Enum):
    MOUNTAIN=0
    VALLEY=1 
def addFold(strip,mv,angle):
    strip.append(Fold(mv,angle))
    return strip
 
def render (strip, striplength, stripwidth):
    d = draw.Drawing(1000,stripwidth)
    for element in strip:
        if type(element) is Plain:
            striplength += element.length
        if type(element) is Fold:
            l = element.getLength(stripwidth)
            x1 = striplength 
            x2 = striplength+abs(l) 
            if l <0: x1,x2 = x2,x1
            d.append(draw.Line(x1,0,x2,stripwidth,stroke='red' if element.mv is MV.MOUNTAIN else 'blue', fill='none'))    
            striplength += abs(l)
    d.append(draw.Rectangle(0, 0, striplength, stripwidth,stroke='red', fill='none'))        
    d.saveSvg('test.svg')
    d.rasterize()
    return d

strip = addFold(strip,MV.MOUNTAIN, -45)
strip = addFold(strip,MV.MOUNTAIN, 45)
strip = addPlain(strip,100)
strip = addFold(strip,MV.VALLEY, -30)

render (strip, striplength, stripwidth)

In [4]:
def addCornerGadget (strip,mv):
    strip = addFold(strip,mv,45)
    strip = addFold(strip,mv,-45)
    return strip
strip = addCornerGadget(strip,MV.MOUNTAIN)
render (strip, striplength, stripwidth)

In [5]:
class Direction(Enum):
    L=-1
    R=1 
def addCornerGadget (strip,mv,d):
    strip = addFold(strip,mv,45*d.value)
    strip = addFold(strip,mv,-45*d.value)
    return strip
strip = addCornerGadget(strip,MV.MOUNTAIN,Direction.R)
strip = addPlain(strip,100)
strip = addCornerGadget(strip,MV.MOUNTAIN,Direction.L)
render (strip, striplength, stripwidth)

In [14]:
import numpy

def getDistance(a,b):
    return numpy.linalg.norm(numpy.array(a) - numpy.array(b))
def cosinelaw(adj_side1,adj_side2,opp_side):
    return math.acos((adj_side1 ** 2 + adj_side2 ** 2 - opp_side **2) / (2*adj_side1 * adj_side2))
def calculateAngles(a,b,c):
    ab = getDistance(a,b)
    bc = getDistance(b,c)
    ac = getDistance(a,c)
    alpha = cosinelaw(ab,ac,bc)
    beta = cosinelaw(ab,bc,ac)
    gamma = cosinelaw(ac,bc,ab)
    return alpha,beta,gamma
def getCorners(t):
    return t[:3], t[3:6], t [6:]
def cornerExtension(gamma,beta,direction,strip_width):
    return math.tan(0.5*math.pi - (gamma if direction is Direction.L else beta))*strip_width
def addTriangleGadget (strip,mv,strip_width,triangle):
    a,b,c = getCorners(triangle)
    alpha,beta,gamma = calculateAngles(a,b,c)
    height = getDistance(a,b) * math.sin(beta)
    d = Direction.R
    strip = addPlain(strip, cornerExtension(gamma, beta,d,strip_width)) # start with a plain stip
    strip = addCornerGadget(strip,mv,d)  # fold a corner
    y=0
    while(y < height):
        d = Direction.R if d is Direction.L else Direction.L
        strip = addPlain(strip,(getDistance(b,c)*y)/height)
        strip = addPlain(strip, cornerExtension(gamma,beta,d,strip_width))
        strip = addCornerGadget(strip,mv,d)
        y+=strip_width
    return strip
strip = []
stripwidth = 10
strip = addTriangleGadget(strip,MV.MOUNTAIN,stripwidth,[ 50, -20, 200,  50, -20, 0,  50, 180,  0]) 
render (strip, striplength, stripwidth)

In [15]:
import stl
from stl import mesh as m

#load the mesh
my_mesh = m.Mesh.from_file('cube.stl')
strip = []
stripwidth = 3
for triangle in my_mesh.points:
    strip = addTriangleGadget(strip,MV.MOUNTAIN,stripwidth,my_mesh.points[0])    

render (strip, striplength, stripwidth)